In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

clf = pickle.load(open('out/model_1.pkl', 'rb'))
clf

In [ ]:
feature_importance = clf.feature_importances_
sorted_idx = np.argsort(-feature_importance)[:20]

pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(12, 6))

plt.barh(pos, feature_importance[sorted_idx], align='center')
# plt.yticks(pos, np.array(diabetes.feature_names)[sorted_idx])
plt.title('Feature Importance (MDI)')

In [ ]:
sorted_idx

In [ ]:
np.sort(-feature_importance)